# Creating analytics tables with Pandas

In [1]:
import pandas as pd
import numpy as np

import sqlalchemy as sq
from sqlalchemy.orm import Session
from sqlalchemy_utils import database_exists, create_database

import csv
from io import StringIO

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
#connect to postgres db
engine = sq.create_engine('postgresql://jupyterdb_user:jupyterdb_user_123@db:5432/analytics_tables')
if not database_exists(engine.url):
    create_database(engine.url)

print(database_exists(engine.url), engine.url)

True postgresql://jupyterdb_user:***@db:5432/analytics_tables


In [3]:
#look at all tables in the database
insp = sq.inspect(engine)
schemas = insp.get_schema_names()
for schema in schemas:
    print("schema: ", schema)
    for table_name in insp.get_table_names(schema=schema):
        print("table_name: ",table_name)
        for column in insp.get_columns(table_name, schema=schema):
            print("Column: ", column)

schema:  information_schema
table_name:  sql_parts
Column:  {'name': 'feature_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column:  {'name': 'feature_name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column:  {'name': 'is_supported', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column:  {'name': 'is_verified_by', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column:  {'name': 'comments', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
table_name:  sql_implementation_info
Column:  {'name': 'implementation_info_id', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
Column:  {'name': 'implementation_info_name', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': False, 'comment

## Define functions Select and Insert

In [4]:
def select(sql: str, engine = engine) -> pd.DataFrame:
    with engine.connect() as connection:
        df_sql = pd.DataFrame(connection.execute(sq.text(sql)))
    return df_sql

In [5]:
# from https://stackoverflow.com/questions/23103962/how-to-write-dataframe-to-postgres-table

def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

## Prepare Data from csv

In [6]:
df = pd.read_csv('data/data.csv', encoding= 'cp1252' , header=0)

In [7]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.390,17850.000,United Kingdom


In [8]:
df.shape

(541909, 8)

In [9]:
df.dtypes

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object

In [10]:
df.columns.str.lower()

Index(['invoiceno', 'stockcode', 'description', 'quantity', 'invoicedate',
       'unitprice', 'customerid', 'country'],
      dtype='object')

In [11]:
df.columns = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unitprice', 'customer_id', 'country']

In [12]:
df.head()

,invoice_no,stock_code,description,quantity,invoice_date,unitprice,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.390,17850.000,United Kingdom


In [13]:
df['invoice_date'] = pd.to_datetime(df['invoice_date'], format = '%m/%d/%Y %H:%M')

In [14]:
df.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unitprice              float64
customer_id            float64
country                 object
dtype: object

## Create table in postgres and upload data

In [15]:
df.to_sql('ecommerce', 
          engine, index=False,
          if_exists='replace',
          method=psql_insert_copy)

In [16]:
#Test
sql = ''' select * from ecommerce t limit 10 '''
select(sql)

,invoice_no,stock_code,description,quantity,invoice_date,unitprice,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.650,17850.000,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.250,17850.000,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.850,17850.000,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.850,17850.000,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.690,13047.000,United Kingdom


## Generate generate date range


In [17]:
#generate date range
sql = ''' select
t.invoice_date,
date_trunc('month', t.invoice_date) as month
 from ecommerce t
 limit 100
'''
select(sql)

,invoice_date,month
0,2010-12-01 08:26:00,2010-12-01
1,2010-12-01 08:26:00,2010-12-01
2,2010-12-01 08:26:00,2010-12-01
3,2010-12-01 08:26:00,2010-12-01
4,2010-12-01 08:26:00,2010-12-01
...,...,...
95,2010-12-01 09:37:00,2010-12-01
96,2010-12-01 09:37:00,2010-12-01
97,2010-12-01 09:37:00,2010-12-01
98,2010-12-01 09:37:00,2010-12-01


In [18]:
# find the period by dates
sql = ''' select
min(date_trunc('month', t.invoice_date)) as min_month,
max(date_trunc('month', t.invoice_date)) as max_month
from ecommerce t
'''
select(sql)

,min_month,max_month
0,2010-12-01,2011-12-01


In [19]:
# generate months by period
sql = ''' SELECT date_trunc('day', dd):: date as month
FROM generate_series
        (( select
min(date_trunc('month', t.invoice_date)) as min_month
from ecommerce t)
        , ( select
max(date_trunc('month', t.invoice_date)) as max_month
from ecommerce t)
        , '1 day'::interval) dd
'''
select(sql)

,month
0,2010-12-01
1,2010-12-02
2,2010-12-03
3,2010-12-04
4,2010-12-05
...,...
361,2011-11-27
362,2011-11-28
363,2011-11-29
364,2011-11-30


### Replace null values

In [20]:
sql = ''' select

t.*,
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month

from ecommerce t
limit 100

'''
select(sql)

,invoice_no,stock_code,description,quantity,invoice_date,unitprice,customer_id,country,customer_id,month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.550,17850.000,United Kingdom,17850.000,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,17850.000,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.750,17850.000,United Kingdom,17850.000,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,17850.000,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.390,17850.000,United Kingdom,17850.000,2010-12-01
...,...,...,...,...,...,...,...,...,...,...
95,536378,22352,LUNCH BOX WITH CUTLERY RETROSPOT,6,2010-12-01 09:37:00,2.550,14688.000,United Kingdom,14688.000,2010-12-01
96,536378,21212,PACK OF 72 RETROSPOT CAKE CASES,120,2010-12-01 09:37:00,0.420,14688.000,United Kingdom,14688.000,2010-12-01
97,536378,21975,PACK OF 60 DINOSAUR CAKE CASES,24,2010-12-01 09:37:00,0.550,14688.000,United Kingdom,14688.000,2010-12-01
98,536378,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24,2010-12-01 09:37:00,0.550,14688.000,United Kingdom,14688.000,2010-12-01


## Per user find minimum date and month, groupby


In [21]:
sql = ''' select

case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(t.invoice_date) as reg_date,
min(date_trunc('month', t.invoice_date)) as reg_month

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end

limit 100
'''
select(sql)

,customer_id,reg_date,reg_month
0,-1.000,2010-12-01 11:52:00,2010-12-01
1,12346.000,2011-01-18 10:01:00,2011-01-01
2,12347.000,2010-12-07 14:57:00,2010-12-01
3,12348.000,2010-12-16 19:09:00,2010-12-01
4,12349.000,2011-11-21 09:51:00,2011-11-01
...,...,...,...
95,12461.000,2011-07-08 10:23:00,2011-07-01
96,12462.000,2011-02-09 14:44:00,2011-02-01
97,12463.000,2011-04-12 12:47:00,2011-04-01
98,12464.000,2011-02-03 13:30:00,2011-02-01


In [22]:
# check that we have not lost users

sql = ''' select
count(distinct case when t.customer_id is null then -1 else t.customer_id end)
from ecommerce t
'''
select(sql)

,count
0,4373


In [23]:
# check the complete select
sql = '''
with users as (

select

case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(t.invoice_date) as reg_date,
min(date_trunc('month', t.invoice_date)) as reg_month

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end)

select count(distinct t.customer_id) from users t
limit 100
'''
select(sql)

,count
0,4373


In [24]:
#find min month per user

sql = '''select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end


limit 100
'''
select(sql)

,customer_id,reg_month,reg_date
0,-1.000,2010-12-01,2010-12-01 11:52:00
1,12346.000,2011-01-01,2011-01-18 10:01:00
2,12347.000,2010-12-01,2010-12-07 14:57:00
3,12348.000,2010-12-01,2010-12-16 19:09:00
4,12349.000,2011-11-01,2011-11-21 09:51:00
...,...,...,...
95,12461.000,2011-07-01,2011-07-08 10:23:00
96,12462.000,2011-02-01,2011-02-09 14:44:00
97,12463.000,2011-04-01,2011-04-12 12:47:00
98,12464.000,2011-02-01,2011-02-03 13:30:00


### Cross join tables users and dates

In [25]:
sql = '''
with users as (
    select
    case when t.customer_id is null then -1 else t.customer_id end as customer_id,
    min(t.invoice_date) as reg_date,
    min(date_trunc('month', t.invoice_date)) as reg_month

    from ecommerce t
    group by
    case when t.customer_id is null then -1 else t.customer_id end
    ),

dates as (
    SELECT date_trunc('day', dd):: date as month
    FROM generate_series
            (( select
    min(date_trunc('month', t.invoice_date)) as min_month
    from ecommerce t)
            , ( select
    max(date_trunc('month', t.invoice_date)) as max_month
    from ecommerce t)
            , '1 day'::interval) dd
)

select t.month, u.* from dates t
join users u on 1=1
where u.customer_id = 12648.0
order by t.month
limit 100

'''
select(sql)

,month,customer_id,reg_date,reg_month
0,2010-12-01,12648.000,2011-07-08 11:55:00,2011-07-01
1,2010-12-02,12648.000,2011-07-08 11:55:00,2011-07-01
2,2010-12-03,12648.000,2011-07-08 11:55:00,2011-07-01
3,2010-12-04,12648.000,2011-07-08 11:55:00,2011-07-01
4,2010-12-05,12648.000,2011-07-08 11:55:00,2011-07-01
...,...,...,...,...
95,2011-03-06,12648.000,2011-07-08 11:55:00,2011-07-01
96,2011-03-07,12648.000,2011-07-08 11:55:00,2011-07-01
97,2011-03-08,12648.000,2011-07-08 11:55:00,2011-07-01
98,2011-03-09,12648.000,2011-07-08 11:55:00,2011-07-01


### Remove extra months before registration date


In [26]:
# cross join
sql = '''
with users as (
    select
    case when t.customer_id is null then -1 else t.customer_id end as customer_id,
    min(t.invoice_date) as reg_date,
    min(date_trunc('month', t.invoice_date)) as reg_month

    from ecommerce t
    group by
    case when t.customer_id is null then -1 else t.customer_id end
    ),

dates as (
    SELECT date_trunc('day', dd):: date as month
    FROM generate_series
            (( select
    min(date_trunc('month', t.invoice_date)) as min_month
    from ecommerce t)
            , ( select
    max(date_trunc('month', t.invoice_date)) as max_month
    from ecommerce t)
            , '1 day'::interval) dd
)

select t.month, u.* from dates t
join users u on t.month >= u.reg_month
where u.customer_id = 12648.0
order by t.month
limit 100

'''
select(sql)

,month,customer_id,reg_date,reg_month
0,2011-07-01,12648.000,2011-07-08 11:55:00,2011-07-01
1,2011-07-02,12648.000,2011-07-08 11:55:00,2011-07-01
2,2011-07-03,12648.000,2011-07-08 11:55:00,2011-07-01
3,2011-07-04,12648.000,2011-07-08 11:55:00,2011-07-01
4,2011-07-05,12648.000,2011-07-08 11:55:00,2011-07-01
...,...,...,...,...
95,2011-10-04,12648.000,2011-07-08 11:55:00,2011-07-01
96,2011-10-05,12648.000,2011-07-08 11:55:00,2011-07-01
97,2011-10-06,12648.000,2011-07-08 11:55:00,2011-07-01
98,2011-10-07,12648.000,2011-07-08 11:55:00,2011-07-01


## Create template

In [27]:
sql = '''
with users as (
    select
    case when t.customer_id is null then -1 else t.customer_id end as customer_id,
    min(t.invoice_date) as reg_date,
    min(date_trunc('month', t.invoice_date)) as reg_month

    from ecommerce t
    group by
    case when t.customer_id is null then -1 else t.customer_id end
    ),

dates as (
    SELECT date_trunc('day', dd):: date as month
    FROM generate_series
            (( select
    min(date_trunc('month', t.invoice_date)) as min_month
    from ecommerce t)
            , ( select
    max(date_trunc('month', t.invoice_date)) as max_month
    from ecommerce t)
            , '1 day'::interval) dd
),

template as (
    select t.month, u.* from dates t
    join users u on t.month >= u.reg_month
 
)

select * from template t
where t.customer_id = 12648.0
order by t.month
limit 100

'''
select(sql)

,month,customer_id,reg_date,reg_month
0,2011-07-01,12648.000,2011-07-08 11:55:00,2011-07-01
1,2011-07-02,12648.000,2011-07-08 11:55:00,2011-07-01
2,2011-07-03,12648.000,2011-07-08 11:55:00,2011-07-01
3,2011-07-04,12648.000,2011-07-08 11:55:00,2011-07-01
4,2011-07-05,12648.000,2011-07-08 11:55:00,2011-07-01
...,...,...,...,...
95,2011-10-04,12648.000,2011-07-08 11:55:00,2011-07-01
96,2011-10-05,12648.000,2011-07-08 11:55:00,2011-07-01
97,2011-10-06,12648.000,2011-07-08 11:55:00,2011-07-01
98,2011-10-07,12648.000,2011-07-08 11:55:00,2011-07-01


## Aggregate revenue data

In [28]:
sql = '''
select * from ecommerce t
where t.customer_id = '12648'
limit 10
'''
select(sql)

,invoice_no,stock_code,description,quantity,invoice_date,unitprice,customer_id,country
0,559421,22962,JAM JAR WITH PINK LID,12,2011-07-08 11:55:00,0.850,12648.000,Germany
1,559421,47590B,PINK HAPPY BIRTHDAY BUNTING,3,2011-07-08 11:55:00,5.450,12648.000,Germany
2,559421,47590A,BLUE HAPPY BIRTHDAY BUNTING,3,2011-07-08 11:55:00,5.450,12648.000,Germany
3,559421,20674,GREEN POLKADOT BOWL,8,2011-07-08 11:55:00,1.250,12648.000,Germany
4,559421,20675,BLUE POLKADOT BOWL,8,2011-07-08 11:55:00,1.250,12648.000,Germany
5,559421,20677,PINK POLKADOT BOWL,8,2011-07-08 11:55:00,1.250,12648.000,Germany
6,559421,21239,PINK POLKADOT CUP,8,2011-07-08 11:55:00,0.850,12648.000,Germany
7,559421,21240,BLUE POLKADOT CUP,8,2011-07-08 11:55:00,0.850,12648.000,Germany
8,559421,21245,GREEN POLKADOT PLATE,8,2011-07-08 11:55:00,1.690,12648.000,Germany
9,559421,21244,BLUE POLKADOT PLATE,16,2011-07-08 11:55:00,1.690,12648.000,Germany


In [29]:
sql = '''
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue


from ecommerce t
where t.customer_id = '12648'

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)

'''
select(sql)

,customer_id,month,revenue
0,12648.000,2011-07-01,318.050


In [30]:
sql = '''
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue

from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)

'''
select(sql)

,customer_id,month,revenue
0,-1.000,2010-12-01,194353.000
1,-1.000,2011-01-01,84925.880
2,-1.000,2011-02-01,61516.500
3,-1.000,2011-03-01,103302.470
4,-1.000,2011-04-01,67159.270
...,...,...,...
13683,18283.000,2011-10-01,114.650
13684,18283.000,2011-11-01,651.560
13685,18283.000,2011-12-01,208.000
13686,18287.000,2011-05-01,765.280


In [31]:
# check by control revenue sum
sql = '''
select
sum(t.quantity * t.unitprice) as revenue
from ecommerce t
'''
select(sql)

,revenue
0,9747747.934


In [32]:
sql = '''
with user_month as (
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue

from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
)
select sum(t.revenue) from user_month t
'''
select(sql)

,sum
0,9747747.934


verification completed successfully


## Join user_month and get an analytical table

In [33]:
sql = '''
with users as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end
),

dates as
(
SELECT date_trunc('day', dd):: date as month
FROM generate_series
        ((select min(date_trunc('month', t.invoice_date)) from ecommerce t)
        ,(select max(date_trunc('month', t.invoice_date)) from ecommerce t)
        , '1 month'::interval) dd
),

templete as
(
select t.month, u.* from dates t
join users u on 1=1 and t.month >= u.reg_month
),

user_month as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue

from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
)

select * from templete t
left join user_month u on t.customer_id = u.customer_id and t.month = u.month
where t.customer_id = '12648'
order by t.month
'''
select(sql, engine)

,month,customer_id,reg_month,reg_date,customer_id,month,revenue
0,2011-07-01,12648.000,2011-07-01,2011-07-08 11:55:00,12648.000,2011-07-01,318.050
1,2011-08-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN
2,2011-09-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN
3,2011-10-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN
4,2011-11-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN
5,2011-12-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN


fix the names


In [34]:
sql = '''
with users as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end
),

dates as
(
SELECT date_trunc('day', dd):: date as month
FROM generate_series
        ((select min(date_trunc('month', t.invoice_date)) from ecommerce t)
        ,(select max(date_trunc('month', t.invoice_date)) from ecommerce t)
        , '1 month'::interval) dd
),

templete as
(
select t.month, u.* from dates t
join users u on 1=1 and t.month >= u.reg_month
),

user_month as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue_sum,
sum(t.quantity) as quantity_sum


from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
)

select * from templete t
left join user_month u on t.customer_id = u.customer_id and t.month = u.month
where t.customer_id = '12648'
order by t.month
'''
select(sql, engine)

,month,customer_id,reg_month,reg_date,customer_id,month,revenue_sum,quantity_sum
0,2011-07-01,12648.000,2011-07-01,2011-07-08 11:55:00,12648.000,2011-07-01,318.050,160
1,2011-08-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN,None
2,2011-09-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN,None
3,2011-10-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN,None
4,2011-11-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN,None
5,2011-12-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,NaT,NaN,None


In [35]:
sql = '''
with users as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end
),

dates as
(
SELECT date_trunc('day', dd):: date as month
FROM generate_series
        ((select min(date_trunc('month', t.invoice_date)) from ecommerce t)
        ,(select max(date_trunc('month', t.invoice_date)) from ecommerce t)
        , '1 month'::interval) dd
),

templete as
(
select t.month, u.* from dates t
join users u on 1=1 and t.month >= u.reg_month
),

user_month as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue_sum,
sum(t.quantity) as quantity_sum


from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
)

select t.*, u.revenue_sum, u.quantity_sum from templete t
left join user_month u on t.customer_id = u.customer_id and t.month = u.month
where t.customer_id = '12648'
order by t.month
'''
select(sql)

,month,customer_id,reg_month,reg_date,revenue_sum,quantity_sum
0,2011-07-01,12648.000,2011-07-01,2011-07-08 11:55:00,318.050,160
1,2011-08-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None
2,2011-09-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None
3,2011-10-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None
4,2011-11-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None
5,2011-12-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None


## Add unique products

In [36]:
sql = '''
with users as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end
),

dates as
(
SELECT date_trunc('day', dd):: date as month
FROM generate_series
        ((select min(date_trunc('month', t.invoice_date)) from ecommerce t)
        ,(select max(date_trunc('month', t.invoice_date)) from ecommerce t)
        , '1 month'::interval) dd
),

templete as
(
select t.month, u.* from dates t
join users u on 1=1 and t.month >= u.reg_month
),

user_month as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue_sum,
sum(t.quantity) as quantity_sum,
count(distinct t.description) as description_cnt_distinct

from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
)

select t.*,
u.revenue_sum,
u.quantity_sum,
u.description_cnt_distinct,
1 as user,
case when u.revenue_sum is not null then 1 else 0 end as active,
case when t.month = t.reg_month then 1 else 0 end as new


from templete t
left join user_month u on t.customer_id = u.customer_id and t.month = u.month
where t.customer_id = '12648'
order by t.month
'''
select(sql)

,month,customer_id,reg_month,reg_date,revenue_sum,quantity_sum,description_cnt_distinct,user,active,new
0,2011-07-01,12648.000,2011-07-01,2011-07-08 11:55:00,318.050,160,21.000,1,1,1
1,2011-08-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None,NaN,1,0,0
2,2011-09-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None,NaN,1,0,0
3,2011-10-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None,NaN,1,0,0
4,2011-11-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None,NaN,1,0,0
5,2011-12-01,12648.000,2011-07-01,2011-07-08 11:55:00,NaN,None,NaN,1,0,0


## Create a monthly analytics report

In [37]:
sql = '''
with users as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end
),

dates as
(
SELECT date_trunc('day', dd):: date as month
FROM generate_series
        ((select min(date_trunc('month', t.invoice_date)) from ecommerce t)
        ,(select max(date_trunc('month', t.invoice_date)) from ecommerce t)
        , '1 month'::interval) dd
),

templete as
(
select t.month, u.* from dates t
join users u on 1=1 and t.month >= u.reg_month
),

user_month as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue_sum,
sum(t.quantity) as quantity_sum,
count(distinct t.description) as description_cnt_distinct

from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
),

analytical_table as 
(
select t.*,
u.revenue_sum,
u.quantity_sum,
u.description_cnt_distinct,
1 as user,
case when u.revenue_sum is not null then 1 else 0 end as active,
case when t.month = t.reg_month then 1 else 0 end as new


from templete t
left join user_month u on t.customer_id = u.customer_id and t.month = u.month
)

select
t.month,
sum(t.user) as user,
sum(t.active) as active,
avg(t.active) as active_pct,
sum(t.new) as new,
sum(t.revenue_sum) as revenue



from analytical_table t
group by t.month
order by t.month

'''
select(sql, engine)

,month,user,active,active_pct,new,revenue
0,2010-12-01,949,949,1.00000000000000000000,949,748957.020
1,2011-01-01,1370,784,0.57226277372262773723,421,560000.260
2,2011-02-01,1750,799,0.45657142857142857143,380,498062.650
3,2011-03-01,2190,1021,0.46621004566210045662,440,683267.080
4,2011-04-01,2489,900,0.36159100040176777822,299,493207.121
5,2011-05-01,2768,1080,0.39017341040462427746,279,723333.510
6,2011-06-01,3003,1052,0.35031635031635031635,235,691123.120
7,2011-07-01,3194,994,0.31120851596743894803,191,681300.111
8,2011-08-01,3361,981,0.29187741743528711693,167,682680.510
9,2011-09-01,3659,1303,0.35610822629133643072,298,1019687.622


### Check revenue sum

In [38]:
sql = '''
with users as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end
),

dates as
(
SELECT date_trunc('day', dd):: date as month
FROM generate_series
        ((select min(date_trunc('month', t.invoice_date)) from ecommerce t)
        ,(select max(date_trunc('month', t.invoice_date)) from ecommerce t)
        , '1 month'::interval) dd
),

templete as
(
select t.month, u.* from dates t
join users u on 1=1 and t.month >= u.reg_month
),

user_month as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue_sum,
sum(t.quantity) as quantity_sum,
count(distinct t.description) as description_cnt_distinct

from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
),

analytical_table as 
(
select t.*,
u.revenue_sum,
u.quantity_sum,
u.description_cnt_distinct,
1 as user,
case when u.revenue_sum is not null then 1 else 0 end as active,
case when t.month = t.reg_month then 1 else 0 end as new


from templete t
left join user_month u on t.customer_id = u.customer_id and t.month = u.month
)

select
t.month,
sum(t.user) as user,
sum(t.active) as active_sum,
avg(t.active) as active_pct_sum,
sum(t.new) as new_sum,
sum(t.revenue_sum) as revenue_sum,
avg(t.revenue_sum) as quantity_avg,
avg(t.description_cnt_distinct) as description_cnt_distinct_avg



from analytical_table t
group by t.month
order by t.month

'''
select(sql)

,month,user,active_sum,active_pct_sum,new_sum,revenue_sum,quantity_avg,description_cnt_distinct_avg
0,2010-12-01,949,949,1.00000000000000000000,949,748957.020,789.207,27.1464699683877766
1,2011-01-01,1370,784,0.57226277372262773723,421,560000.260,714.286,28.1900510204081633
2,2011-02-01,1750,799,0.45657142857142857143,380,498062.650,623.358,25.7133917396745932
3,2011-03-01,2190,1021,0.46621004566210045662,440,683267.080,669.214,26.8334965719882468
4,2011-04-01,2489,900,0.36159100040176777822,299,493207.121,548.008,25.6844444444444444
5,2011-05-01,2768,1080,0.39017341040462427746,279,723333.510,669.753,26.0685185185185185
6,2011-06-01,3003,1052,0.35031635031635031635,235,691123.120,656.961,26.1311787072243346
7,2011-07-01,3194,994,0.31120851596743894803,191,681300.111,685.413,27.2002012072434608
8,2011-08-01,3361,981,0.29187741743528711693,167,682680.510,695.903,27.7726809378185525
9,2011-09-01,3659,1303,0.35610822629133643072,298,1019687.622,782.569,30.2594013814274751


In [39]:
t_1 = select(sql)

In [40]:
t_1['revenue_sum'].sum()

9747747.934000026

In [41]:
sql = '''
select
sum(t.quantity * t.unitprice) as revenue_sum

from ecommerce t
'''
select(sql, engine)

,revenue_sum
0,9747747.934


check done

## Final analytical report


In [42]:
sql = '''
with users as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
min(date_trunc('month', t.invoice_date)) as reg_month,
min(t.invoice_date) as reg_date

from ecommerce t
group by
case when t.customer_id is null then -1 else t.customer_id end
),

dates as
(
SELECT date_trunc('day', dd):: date as month
FROM generate_series
        ((select min(date_trunc('month', t.invoice_date)) from ecommerce t)
        ,(select max(date_trunc('month', t.invoice_date)) from ecommerce t)
        , '1 month'::interval) dd
),

templete as
(
select t.month, u.* from dates t
join users u on 1=1 and t.month >= u.reg_month
),

user_month as 
(
select
case when t.customer_id is null then -1 else t.customer_id end as customer_id,
date_trunc('month', t.invoice_date) as month,
sum(t.quantity * t.unitprice) as revenue_sum,
sum(t.quantity) as quantity_sum,
count(distinct t.description) as description_cnt_distinct

from ecommerce t

group by
case when t.customer_id is null then -1 else t.customer_id end,
date_trunc('month', t.invoice_date)
),

analytical_table as 
(
select t.*,
u.revenue_sum,
u.quantity_sum,
u.description_cnt_distinct,
1 as user,
case when u.revenue_sum is not null then 1 else 0 end as active,
case when t.month = t.reg_month then 1 else 0 end as new


from templete t
left join user_month u on t.customer_id = u.customer_id and t.month = u.month
)

select
t.month,
sum(t.user) as user,
sum(t.active) as active_sum,
avg(t.active) as active_pct_sum,
sum(t.new) as new_sum,
sum(t.revenue_sum) as revenue_sun,
avg(t.revenue_sum) as quantity_avg,
avg(t.description_cnt_distinct) as description_cnt_distinct_avg



from analytical_table t
group by t.month
order by t.month

'''
select(sql, engine)

,month,user,active_sum,active_pct_sum,new_sum,revenue_sun,quantity_avg,description_cnt_distinct_avg
0,2010-12-01,949,949,1.00000000000000000000,949,748957.020,789.207,27.1464699683877766
1,2011-01-01,1370,784,0.57226277372262773723,421,560000.260,714.286,28.1900510204081633
2,2011-02-01,1750,799,0.45657142857142857143,380,498062.650,623.358,25.7133917396745932
3,2011-03-01,2190,1021,0.46621004566210045662,440,683267.080,669.214,26.8334965719882468
4,2011-04-01,2489,900,0.36159100040176777822,299,493207.121,548.008,25.6844444444444444
5,2011-05-01,2768,1080,0.39017341040462427746,279,723333.510,669.753,26.0685185185185185
6,2011-06-01,3003,1052,0.35031635031635031635,235,691123.120,656.961,26.1311787072243346
7,2011-07-01,3194,994,0.31120851596743894803,191,681300.111,685.413,27.2002012072434608
8,2011-08-01,3361,981,0.29187741743528711693,167,682680.510,695.903,27.7726809378185525
9,2011-09-01,3659,1303,0.35610822629133643072,298,1019687.622,782.569,30.2594013814274751
